In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import random
import urllib
from pathlib import Path
import pandas as pd
from functools import partial
import re

In [2]:
class InstagramBot():
    """
    Bot used to scrape information from instagram and perform common features including liking posts
    
    To avoid getting your account banned during troubleshooting, make sure to reinitialize
    your bot with your current browser rather than opening up a new one each time.
    
    
    
    """
    def __init__(self, email, password, browser = None):
        self.browser = webdriver.Chrome() if browser is None else browser
        self.email = email
        self.password = password
        
        
    def signIn(self):
        self.browser.get('https://www.instagram.com/accounts/login/')
        time.sleep(2)
        emailInput = self.browser.find_elements_by_css_selector('form input')[0]
        passwordInput = self.browser.find_elements_by_css_selector('form input')[1]
        
        emailInput.send_keys(self.email)
        passwordInput.send_keys(self.password)
        passwordInput.send_keys(Keys.ENTER)
        time.sleep(2)
        
    def followerList(self, user, max_followers = 1000):
        self.browser.get('https://www.instagram.com/' + user)
        
        time.sleep(20)
        allLinks = self.browser.find_elements_by_css_selector('ul li a')

        followLink = None
        for link in allLinks:
            if link.text.endswith('followers'):
                followLink = link
                break

        followLink.click()
        time.sleep(1)
        followersList = self.browser.find_element_by_css_selector('div[role=\'dialog\'] ul')
        numberFollowersList = len(followersList.find_elements_by_css_selector('li'))
        lastFollowersList = 0
        followersList.find_element_by_css_selector('li').click()

        actionChain = webdriver.ActionChains(self.browser)
        while (numberFollowersList< max_followers and lastFollowersList != numberFollowersList):
            followersList.find_elements_by_css_selector('li')[-1].click()
            lastFollowersList = numberFollowersList
            actionChain.key_down(Keys.SPACE).key_up(Keys.SPACE).perform()
            time.sleep(1)
            numberFollowersList = len(followersList.find_elements_by_css_selector('li'))
            #print(numberFollowersList, lastFollowersList)

        followers = []
        for user in followersList.find_elements_by_css_selector('li'):
            userLink = user.find_element_by_css_selector('a').get_attribute('href')
            followers.append(userLink)
            
        return followers
    
    def goto(self, url):
        self.browser.get(url)
        
        
    def check_private(self):
        return bool(self.browser.find_elements_by_xpath("//*[contains(text(), 'This Account is Private')]"))
    
    
    def check_public(self):
        return not self.check_private()
    
    
    def like_post(self):
        like_button = self.browser.find_elements_by_css_selector('span[aria-label=\'Like\']')
        if bool(like_button): 
            like_button[0].click()
            return True
        else:
            return False
            
    def next_post(self):
        next_button = self.browser.find_elements_by_xpath("//*[contains(text(), 'Next')]")
        if bool(next_button): 
            next_button[0].click()
            return True
        else: return False
        
    def first_post(self):
        first_pic = self.browser.find_elements_by_class_name("_9AhH0")
        if bool(first_pic): 
            first_pic[0].click()
            return True
        else: return False
        
        
    def wait_random(self, low = 1, high = 10):
        wait_time = random.randint(low, high)
        time.sleep(wait_time/10)
        
    def check_action_block(self):
        block_button = self.browser.find_elements_by_xpath("//*[contains(text(), 'This action was blocked')]")
        return bool(block_button)
    
    def reset_activity(self):
        active_button = self.browser.find_elements_by_css_selector('svg[aria-label=\'Activity Feed\']')
        try:
            if bool(active_button): 
                active_button[0].click()
                
                self.wait_random(40,60)
                
                close_activity_button = self.browser.find_elements_by_class_name('_8Mwnh')
                close_activity_button[0].click()
                
                return True
        
            else:
                return False
        except Exception as e:
            print (e)
            return False
        
        
        
    def renavigation_check(self, url):
        if url is not None: self.browser.get(url)
        url = self.browser.current_url if url is None else url
        return url


    
    def like_posts_by_user(self, user, num_posts = 3):
        self.browser.get(user)
        try:

            
            self.wait_random(10,30)
            assert self.check_public(), 'Private account'
            self.wait_random(9,15)
            #Check if there is any activity thaqt needs to be cleared (There is a hidden container that prevents clicking)
            if len(self.browser.find_elements_by_class_name("H9zXO")): 
                assert self.reset_activity(), "Problem resetting activity"
                self.wait_random(10, 30)
            assert self.first_post(), 'No first post'
            self.wait_random(15,30)
            assert self.like_post(), 'Already liked'
            self.wait_random(9,20)
            
            for i in range(num_posts-1):
                for _ in range(random.randint(1,3)):
                    assert self.next_post(), 'Last post'
                    self.wait_random(10,40)
                assert self.like_post(), 'Already liked'
                self.wait_random(9,20)
            
            
            actionChain = webdriver.ActionChains(self.browser)
            actionChain.key_down(Keys.ESCAPE).key_up(Keys.ESCAPE).perform()
            
            #self.wait_random(10, 30)
            #assert self.reset_activity(), "Problem resetting activity"
            #self.wait_random(30, 80)
            
        except AssertionError as e:
            print (e)
            if self.check_action_block(): 
                print('Action Blocked!!')
                return 'Block'
            return False
        except Exception as e:
            print('Unknown error', e)
            if self.check_action_block(): 
                print('Action Blocked!!')
                return 'Block'
        finally:
            actionChain = webdriver.ActionChains(self.browser)
            actionChain.key_down(Keys.ESCAPE).key_up(Keys.ESCAPE).perform()
        
        return True
    
    
    """
    Extract a single metric
    """
        
    def get_username(self): return self.browser.find_element_by_class_name("nJAzx").text
        
    def get_likes(self):
        likes_span = self.browser.find_element_by_class_name('Nm9Fw')
        likes = int(likes_span.find_element_by_tag_name('span').text.replace(',', ''))
        return likes
    
    def get_post_time(self): return self.browser.find_element_by_class_name('_1o9PC').get_attribute('datetime')
    
    def get_alt_text(self): return self.browser.find_element_by_class_name('FFVAD').get_attribute('alt')
    
    def get_comment(self, num): 
        try:
            return self.browser.find_elements_by_xpath("//div[@class='C4VMK']/span[@class='']")[num].text
        except:
            return None
    
    def get_tags(self, separator = ' '):
        comment1, comment2 = self.get_comment(0), self.get_comment(1)
        comments = comment1 if comment2 is None else comment1+comment2
        tags = re.findall('#[A-Za-z]+ ', comments)
        return separator.join(tags)
            
        
        

    """
    Extract groups of information found in the same location and return a dict
    """
    
    
    def collect_user_metrics(self, username = None):
        if username is not None: 
            self.browser.get(r'https://www.instagram.com/'+ username)
        time.sleep(0.5)
        user_elements = self.browser.find_elements_by_class_name('g47SY')
        num_posts = user_elements[0].text.replace(',', '')
        followers = user_elements[1].get_attribute('title').replace(',', '')
        following = user_elements[2].text.replace(',', '')
        return {'num_posts': num_posts,
               'followers': followers,
               'following': following}
    
    
    def collect_post_metrics(self):
        post_info = {}
        metrics = {'likes': self.get_likes,
                  'posttime': self.get_post_time, 
                  'alt-text': self.get_alt_text,
                  'caption': partial(self.get_comment, 0), 
                  'tags': self.get_tags}
        for feature, value in metrics.items():
            try: post_info[feature] = value()
            except: print("Couldn't collect", feature)
        return post_info
    
    
    
    def scrape_post(self, url = None, include_post = True, include_user = True):
        url = self.renavigation_check(url)
        
        time.sleep(3)
        post_info = self.collect_post_metrics() if include_post else {} 
        
        
        time.sleep(1)
        user_info = self.collect_user_metrics() if include_user else {}
        
    
        all_info = {**post_info, **user_info}
        return all_info
    
    
   
    """
    Download Posts
    """
    
    def get_post_id(self, url): return url.split(r'/')[-2]
        
    
    
    def download_post(self, url = None, output_dir = None):
        #Option of using the current page, or navigating to a specific url
        output_dir = Path('output') if output_dir is None else output_dir
        output_dir.mkdir(exist_ok=True)
        url = self.renavigation_check(url)
        time.sleep(2)
            
        try:
            download_link = self.browser.find_elements_by_class_name('FFVAD')[0].get_attribute('src')
            urllib.request.urlretrieve(download_link, output_dir/(url.split(r'/')[-2] + '.jpg'))
        except:
            print('Couldnt print this one: ', url)
        return True






    """
    """
    
    def fill_in_dataframe(self, df, starting_position = 0, max_posts = None, include_post = True, include_user = True):
        self.current_dataframe = df
        for (index, row) in df.iloc[starting_position:,:].iterrows(): 
            print("Current getting index ", index, "Username", df.at[index, 'username'])
            url = df.loc[index,'Links']
            self.browser.get(url)
            time.sleep(1)
            if not self.browser.find_elements_by_class_name('error-container'):
            



                for (key, value) in self.scrape_post(include_post = include_post, include_user = include_user).items():

                        df.at[index, key] = value
                        
                time.sleep(2)
                self.download_post(url)
                
                
                if index%20 == 0: df.to_csv('temp_output.csv')
            #except Exception as e: print(f'Exception: {e} at {url}')

                

            if max_posts: 
                if index>max_posts: break
            
        return df
    
    
    def export_dataframe(self, output_name = 'output.csv'):
        self.current_dataframe.to_csv(output_name)
        
        
        
    def load_dataframe(self, file_name = 'output.csv'):
        return pd.read_csv(file_name)
    
    
    
    def list_of_posts_from_user(self):
        actionChain = webdriver.ActionChains(self.browser)
        all_links = set()
        old_len = -1
        while old_len!=len(all_links):
            old_len = len(all_links)
            actionChain.key_down(Keys.END).key_up(Keys.END).perform()
            time.sleep(2)
            links = self.browser.find_elements_by_xpath('//div[@class="v1Nh3 kIKUG  _bz0w"]/a')
            for element in links:
                try:
                    href = element.get_attribute('href')
                    if r'/p/' in href and href not in all_links: all_links.add(href)
                    #print('Got link')
                except Exception as e:
                    print('Exception', e)
                    
        user_post_list = list(all_links)
        return user_post_list
    
    def gather_posts_from_user(self, username):
        """
        Note: this approach find all posts first, then scrapes through them
        An alternate approach involves going to the first post, then gradually iterating through them
        
        """
        self.browser.get(r'https://www.instagram.com/'+ username)
        user_info = self.collect_user_metrics()
        
        post_list = self.list_of_posts_from_user()
        new_df = pd.DataFrame({'Links':post_list})
        for key, value in user_info.items():
            #new_df.assign(key=value)
            new_df[key] = value
            
        new_df['username'] = username
        
        return new_df
    
    def concat_dataframes(self, old, new): return pd.concat([old, new], ignore_index = True, sort = True)
    
    def add_links_to_dataframe_from_user(self, df, username):
        new_df = self.gather_posts_from_user(username)
        return self.concat_dataframes(df, new_df)
    
    
    def posts_from_list_of_users(self, user_list):
        df = pd.DataFrame()
        for user in user_list:
            df = self.add_links_to_dataframe_from_user(df, user)
            
        return df
    
    
    def extract_user_list_from_dataframe(self, df, filters = []):
        mod_df = df
        for (parameter, minimum) in filters:
            mod_df = mod_df[mod_df[parameter]>minimum]
            
        return list(mod_df['username'])
        
        
    
    

        
        
            
        
        

In [3]:
#IB = InstagramBot('webscraper100', 'ScrapingTheWeb100')

In [30]:
IB = InstagramBot('webscraper100', 'ScrapingTheWeb100', IB.browser)

In [247]:
user_df = IB.posts_from_list_of_users(['theburgeratti', 'minettatavernny'])

In [257]:
user_df = IB.posts_from_list_of_users(user_list)

In [4]:
user_df = IB.load_dataframe('user_output.csv')

In [5]:
IB.fill_in_dataframe(user_df, include_user = False, starting_position = 1100)

Current getting index  1100 Username komotodo_kc
Current getting index  1101 Username komotodo_kc
Current getting index  1102 Username komotodo_kc
Current getting index  1103 Username komotodo_kc
Couldn't collect likes
Couldn't collect alt-text
Couldnt print this one:  https://www.instagram.com/p/B4YTX-Lp70p/
Current getting index  1104 Username komotodo_kc
Current getting index  1105 Username komotodo_kc
Couldn't collect likes
Couldn't collect alt-text
Couldnt print this one:  https://www.instagram.com/p/B5k4ZWvJ5Xx/
Current getting index  1106 Username komotodo_kc
Current getting index  1107 Username komotodo_kc
Current getting index  1108 Username komotodo_kc
Current getting index  1109 Username komotodo_kc
Current getting index  1110 Username komotodo_kc
Current getting index  1111 Username komotodo_kc
Current getting index  1112 Username komotodo_kc
Current getting index  1113 Username komotodo_kc
Current getting index  1114 Username komotodo_kc
Current getting index  1115 Usernam

Current getting index  1230 Username saltandcurry
Current getting index  1231 Username saltandcurry
Current getting index  1232 Username saltandcurry
Current getting index  1233 Username saltandcurry
Current getting index  1234 Username saltandcurry
Current getting index  1235 Username saltandcurry
Current getting index  1236 Username saltandcurry
Current getting index  1237 Username saltandcurry
Current getting index  1238 Username saltandcurry
Current getting index  1239 Username saltandcurry
Current getting index  1240 Username saltandcurry
Current getting index  1241 Username saltandcurry
Current getting index  1242 Username saltandcurry
Current getting index  1243 Username saltandcurry
Current getting index  1244 Username saltandcurry
Current getting index  1245 Username saltandcurry
Current getting index  1246 Username saltandcurry
Current getting index  1247 Username saltandcurry
Current getting index  1248 Username saltandcurry
Current getting index  1249 Username saltandcurry


KeyboardInterrupt: 

In [6]:
df1 = pd.read_csv('temp_output.csv')

In [7]:
df2 = pd.read_csv('temp_output - Copy (2).csv')

In [8]:
pd.merge(df1, df2)

,Unnamed: 0,Unnamed: 0.1,Links,followers,following,num_posts,username,likes,posttime,alt-text,caption,tags
0,60,60,https://www.instagram.com/p/B6M2wVUHQCl/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
1,61,61,https://www.instagram.com/p/B5DPFtvDAOg/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
2,62,62,https://www.instagram.com/p/B4ncu8IH8Px/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
3,63,63,https://www.instagram.com/p/B6jeGEtnu4T/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
4,64,64,https://www.instagram.com/p/B5jW3qelOh8/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
5,65,65,https://www.instagram.com/p/B5RFEaAFW1-/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
6,66,66,https://www.instagram.com/p/B6ILiwkle9x/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
7,67,67,https://www.instagram.com/p/B5DPHa9DN6z/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
8,68,68,https://www.instagram.com/p/B7Hegebn_eF/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN
9,69,69,https://www.instagram.com/p/B61iHFYHNKW/,3227,886,22,xgiih_,NaN,NaN,NaN,NaN,NaN


In [22]:
for index, row in user_df.loc[5:, :].iterrows():
    print(index)

5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279


1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169


In [252]:
user_list = IB.extract_user_list_from_dataframe(df, [('likes', 50)])

In [51]:
IB.signIn()

In [4]:
df = IB.load_dataframe()

In [234]:
list(df[df['likes']>50]['username'])

['paulawdrodzedocelu',
 'wingsource',
 'miniliciousbyrachelle',
 'xgiih_',
 'rockyfoodreport',
 'gioia.x',
 'gioskitchen_jt',
 'beer_bread_brewery',
 'burgercars',
 'babka_na_keto',
 '_astroceleste',
 'forza986',
 'kokofoodie',
 'gooodchivesonly',
 'lolitateqbars',
 'chillicious_sa',
 'myfudz_wine_diary',
 'srundeberg',
 'gil_eo',
 'epatrizi',
 'nathancaesar',
 'oliviaszauberkessel',
 'novitany',
 'christaosfood',
 'crtdktchn',
 'khanlifts',
 'mr_bonnebouffe',
 'epatrizi',
 'caloriehunter_',
 'bobtimr',
 'giappocastellammare',
 'manhattangrilltrinidad',
 'sanja.littlebakery',
 'took.tig',
 'komotodo_kc',
 '_mr_kt.f_',
 'taqueria_el_poblano',
 'saltandcurry',
 'alaparrillapg',
 'akiraramen_izakaya',
 'sporty_jula',
 'irene_argentieri',
 'toughcookie.yo',
 'wg.tasty',
 'artscreationss',
 '__helena__99',
 'comidasporvida',
 'not.your.vegan.girl',
 'ingoodtaste_',
 'budarezuliano',
 'marielofficial',
 'deep.flavors',
 '_clogram',
 'foodartchefs',
 'marcos_foodblog',
 'salvocraverochef',
 '

In [49]:
IB.fill_in_dataframe(df, starting_position = 100)

IndexError: list index out of range

In [9]:
IB.collect_post_metrics()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span"}
  (Session info: chrome=79.0.3945.117)


In [101]:
IB.collect_user_metrics()

{'num_posts': '4481', 'followers': '168626', 'following': '83'}

In [120]:
IB.get_tags()

Get tags is imcomplete!!!


'#Burgraphy  #burger  #burgers  #burgerlovers  #burgertime  #hamburger  #hamburgers  #cheeseburger  #burgerlife  #cheeseburgers  #burgerporn  #instaburger  #foodporn  #burgerconquest  #bestburger  #bestburgers  #bestburgerintown  #bestburgerever  #bestburgersever  #instafood  #foodstagram  #eeeeeats  #eatfamous '

In [197]:
ind_links = IB.gather_posts_from_user('theburgeratti')

In [199]:
ind_links

,Links,num_posts,followers,following,username
0,https://www.instagram.com/p/B7KpInNhVAu/,4481,168661,83,theburgeratti
1,https://www.instagram.com/p/B7JwBO3h_56/,4481,168661,83,theburgeratti
2,https://www.instagram.com/p/B7TEaw7hmGV/,4481,168661,83,theburgeratti
3,https://www.instagram.com/p/B7IEgtcBOzH/,4481,168661,83,theburgeratti
4,https://www.instagram.com/p/B7B71rQhtOa/,4481,168661,83,theburgeratti
5,https://www.instagram.com/p/B7U7AIIh--V/,4481,168661,83,theburgeratti
6,https://www.instagram.com/p/B7SWGdihts-/,4481,168661,83,theburgeratti
7,https://www.instagram.com/p/B7NN4wLBQGp/,4481,168661,83,theburgeratti
8,https://www.instagram.com/p/B7OEJKDhvol/,4481,168661,83,theburgeratti
9,https://www.instagram.com/p/B7PKTZHBbh1/,4481,168661,83,theburgeratti


In [201]:
pd.concat([df, ind_links], ignore_index = True, sort = True)

,Links,Unnamed: 0,followers,following,likes,num_posts,posttime,username
0,https://www.instagram.com/p/B7UKNmZCVxf/,0.0,392,257,30.0,34,2020-01-14T21:22:33.000Z,gianluca.melillo.80
1,https://www.instagram.com/p/B7UD7UMlLkK/,1.0,404,526,16.0,2253,2020-01-14T20:27:37.000Z,mizz_martha
2,https://www.instagram.com/p/B7UAvb9pO1J/,2.0,1588,367,21.0,87,2020-01-14T19:59:47.000Z,maestrosdelfuego
3,https://www.instagram.com/p/B7UDiEjhA05/,3.0,1541,1456,62.0,94,2020-01-14T20:24:11.000Z,paulawdrodzedocelu
4,https://www.instagram.com/p/B7UB8Y-BgS8/,4.0,51477,1805,2702.0,1081,2020-01-14T20:10:18.000Z,wingsource
5,https://www.instagram.com/p/B7ULwqWCgTQ/,5.0,3372,561,19.0,689,2020-01-14T21:36:04.000Z,fassbender_fancy_official
6,https://www.instagram.com/p/B7UL4IGIKZP/,6.0,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.instagram.com/p/B7UBEZZFz9M/,7.0,1490,651,66.0,318,2020-01-14T20:02:39.000Z,miniliciousbyrachelle
8,https://www.instagram.com/p/B7UIYOhAExm/,8.0,1214,508,17.0,391,2020-01-14T21:06:31.000Z,latandabarmid
9,https://www.instagram.com/p/B7UEy0QHk0S/,9.0,188,342,18.0,210,2020-01-14T20:35:12.000Z,ealingeats


In [162]:
len(ind_links)

36

In [165]:
IB.download_post(ind_links[0])

True

In [136]:
image_links = IB.browser.find_elements_by_xpath('//div[@class="v1Nh3 kIKUG  _bz0w"]/a')

In [137]:
for link in image_links:
    print(link.get_attribute('href'))

https://www.instagram.com/p/B7Wq9gyhAX8/
https://www.instagram.com/p/B7VqWj1BPmf/
https://www.instagram.com/p/B7U7AIIh--V/
https://www.instagram.com/p/B7T89dFhIYc/
https://www.instagram.com/p/B7TEaw7hmGV/
https://www.instagram.com/p/B7SWGdihts-/
https://www.instagram.com/p/B7RWq5iB1Pv/
https://www.instagram.com/p/B7Qf-R3hj3d/
https://www.instagram.com/p/B7PKTZHBbh1/
https://www.instagram.com/p/B7OEJKDhvol/
https://www.instagram.com/p/B7NN4wLBQGp/
https://www.instagram.com/p/B7MTo7uBZGK/
https://www.instagram.com/p/B7LdSTjhtWo/
https://www.instagram.com/p/B7KpInNhVAu/
https://www.instagram.com/p/B7JwBO3h_56/
https://www.instagram.com/p/B7Iv56zBLZi/
https://www.instagram.com/p/B7IEgtcBOzH/
https://www.instagram.com/p/B7HRjo1BSnW/
https://www.instagram.com/p/B7GL6VOB9se/
https://www.instagram.com/p/B7FcP16By58/
https://www.instagram.com/p/B7EjFLShGCf/
https://www.instagram.com/p/B7Dn-R9BZIo/
https://www.instagram.com/p/B7C3uythys_/
https://www.instagram.com/p/B7B71rQhtOa/
https://www.inst

In [ ]:
"//div[@class='C4VMK']/span[@class='']"

In [110]:
re.findall('#[A-Za-z]+ ', comments2)

['#Burgraphy ',
 '#burger ',
 '#burgers ',
 '#burgerlovers ',
 '#burgertime ',
 '#hamburger ',
 '#hamburgers ',
 '#cheeseburger ',
 '#burgerlife ',
 '#cheeseburgers ',
 '#burgerporn ',
 '#instaburger ',
 '#foodporn ',
 '#burgerconquest ',
 '#bestburger ',
 '#bestburgers ',
 '#bestburgerintown ',
 '#bestburgerever ',
 '#bestburgersever ',
 '#instafood ',
 '#foodstagram ',
 '#eeeeeats ',
 '#eatfamous ']

In [17]:
IB.browser.find_element_by_class_name('error-container')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".error-container"}
  (Session info: chrome=79.0.3945.117)


In [66]:
IB.browser.find_elements_by_class_name('ZyFrc')[0].find_elements_by_class_name('C4VMK')

[]

In [76]:
IB.browser.find_elements_by_xpath("//div[@class='C4VMK']/span[@class='']")[0].text

'Consider this a little mid-week pick-me-up. 📷: @bon_nappetit /📍: @hanoi.house #forkyeah\u2060\n.\u2060\n.\u2060\n.\u2060\n.\u2060\n.\u2060\n.\u2060\n#foodbeast #buzzfeast #hanoihouse #huffposttaste #devourpower #spoonfeed #feastagram #foodandwine #cheatday #eatfamous #dailyfoodfeed #feedyoursoull #phaat #newforkcity #eeeeeats #foodpornshare #foodporn #foodgasm #thrillist #nyceats #foodie #forkyeah #nyc #nycfat #noodle #noodleworship #liftingnoodles #pho'

In [5]:
df = pd.read_csv('output.csv')

In [6]:
df

,Unnamed: 0,Links,likes,username,num_posts,followers,following,posttime
0,0,https://www.instagram.com/p/B7UKNmZCVxf/,30.0,gianluca.melillo.80,34.0,392.0,257.0,2020-01-14T21:22:33.000Z
1,1,https://www.instagram.com/p/B7UD7UMlLkK/,16.0,mizz_martha,2253.0,404.0,526.0,2020-01-14T20:27:37.000Z
2,2,https://www.instagram.com/p/B7UAvb9pO1J/,21.0,maestrosdelfuego,87.0,1588.0,367.0,2020-01-14T19:59:47.000Z
3,3,https://www.instagram.com/p/B7UDiEjhA05/,62.0,paulawdrodzedocelu,94.0,1541.0,1456.0,2020-01-14T20:24:11.000Z
4,4,https://www.instagram.com/p/B7UB8Y-BgS8/,2702.0,wingsource,1081.0,51477.0,1805.0,2020-01-14T20:10:18.000Z
5,5,https://www.instagram.com/p/B7ULwqWCgTQ/,19.0,fassbender_fancy_official,689.0,3372.0,561.0,2020-01-14T21:36:04.000Z
6,6,https://www.instagram.com/p/B7UL4IGIKZP/,NaN,NaN,NaN,NaN,NaN,NaN
7,7,https://www.instagram.com/p/B7UBEZZFz9M/,66.0,miniliciousbyrachelle,318.0,1490.0,651.0,2020-01-14T20:02:39.000Z
8,8,https://www.instagram.com/p/B7UIYOhAExm/,17.0,latandabarmid,391.0,1214.0,508.0,2020-01-14T21:06:31.000Z
9,9,https://www.instagram.com/p/B7UEy0QHk0S/,18.0,ealingeats,210.0,188.0,342.0,2020-01-14T20:35:12.000Z


In [74]:
username = IB.browser.find_elements_by_class_name("nJAzx")

In [6]:
IB.browser.get(r'https://www.instagram.com/explore/tags/foodporn/')

In [236]:
download_link = IB.browser.find_elements_by_class_name('FFVAD')[0].get_attribute('src')

In [244]:
output_dir = Path('Output')
output_dir.mkdir(exist_ok=True)
urllib.request.urlretrieve(download_link, output_dir/'test2.jpg')

(WindowsPath('Output/test2.jpg'), <http.client.HTTPMessage at 0x15d9e3830f0>)

In [253]:
    def download_post(self, url = None, output_dir = None):
        #Option of using the current page, or navigating to a specific url
        output_dir = Path('output') if output_dir is None else output_dir
        output_dir.mkdir(exist_ok=True)
        if url is not None: self.browser.get(url)
        else: url = self.browser.current_url
            
            
        download_link = IB.browser.find_elements_by_class_name('FFVAD')[0].get_attribute('src')
        
        urllib.request.urlretrieve(download_link, output_dir/(url.split(r'/')[-2] + '.jpg'))
        
        return True

In [254]:
download_post(IB)

True

In [255]:
df.to_csv('output.csv')

In [57]:
import pandas as pd

In [58]:
df = pd.DataFrame(list(all_links))

In [61]:
df.to_csv('posts.csv')

In [29]:
all_links

{'Hello'}

In [220]:
def get_likes(self):
    likes_span = self.browser.find_element_by_class_name('Nm9Fw')
    likes = int(likes_span.find_element_by_tag_name('span').text.replace(',', ''))
    #print(likes)
    return likes
        
def get_username(self):
    username_div = self.browser.find_element_by_class_name("nJAzx")
    username = username_div.text
    return username

def get_user_information(self):
    user_elements = self.browser.find_elements_by_class_name('g47SY')
    num_posts = user_elements[0].text.replace(',', '')
    followers = user_elements[1].get_attribute('title').replace(',', '')
    following = user_elements[2].text.replace(',', '')
    return num_posts, followers, following

def get_post_time(self):
    post_time = self.browser.find_element_by_class_name('_1o9PC')
    return post_time.get_attribute('datetime')


def get_post_info(self):
    post_info = {}
    post_info['likes'] = get_likes(self)
    post_info['username'] = get_username(self)
    post_info['posttime'] = get_post_time(self)
    self.browser.get(r'https://www.instagram.com/'+post_info['username'])
    time.sleep(0.5)
    user_info = dict(zip(['num_posts','followers','following'], get_user_information(self)))
    post_info = {**post_info, **user_info}
    return post_info

In [222]:
get_post_info(IB)

{'likes': 45,
 'username': 'titi_frenchi',
 'posttime': '2020-01-14T20:13:58.000Z',
 'num_posts': '817',
 'followers': '2391',
 'following': '220'}

In [122]:
print(get_likes(IB), get_username(IB), get_user_information(IB))

1060 color_beatiful_ ('280', '17884', '518')


In [216]:
post_time = IB.browser.find_element_by_class_name('_1o9PC')

In [218]:
post_time.get_attribute('datetime')

'2020-01-09T18:52:56.000Z'

In [ ]:
<time class="_1o9PC Nzb55" datetime="2020-01-09T18:52:56.000Z" title="Jan 9, 2020">5 days ago</time>

In [189]:
for (key, value) in get_post_info(IB).items():
    df.at[5, key] = value
    

In [190]:
df

,Links,Links2,followers,likes,username,num_posts,following
0,https://www.instagram.com/p/B7UKNmZCVxf/,0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.instagram.com/p/B7UAvb9pO1J/,2,NaN,NaN,NaN,NaN,NaN
3,https://www.instagram.com/p/B7UDiEjhA05/,3,NaN,NaN,NaN,NaN,NaN
4,https://www.instagram.com/p/B7UB8Y-BgS8/,4,NaN,NaN,NaN,NaN,NaN
5,https://www.instagram.com/p/B7ULwqWCgTQ/,5,17884.0,1060.0,color_beatiful_,280,518
6,https://www.instagram.com/p/B7UL4IGIKZP/,6,NaN,NaN,NaN,NaN,NaN
7,https://www.instagram.com/p/B7UBEZZFz9M/,7,NaN,NaN,NaN,NaN,NaN
8,https://www.instagram.com/p/B7UIYOhAExm/,8,NaN,NaN,NaN,NaN,NaN
9,https://www.instagram.com/p/B7UEy0QHk0S/,9,NaN,NaN,NaN,NaN,NaN


In [92]:
for el in IB.browser.find_elements_by_class_name('g47SY'):
    print (el.text)

280
17.8k
518


In [204]:
df = pd.DataFrame({'Links': list(all_links)})

In [195]:
df.loc[1,'Links']

'https://www.instagram.com/p/B7UD7UMlLkK/'

In [199]:
get_user_information(IB)

IndexError: list index out of range

,Links,likes,username,num_posts,followers,following,posttime
5,https://www.instagram.com/p/B7ULwqWCgTQ/,19.0,fassbender_fancy_official,689,3372,561,2020-01-14T21:36:04.000Z
6,https://www.instagram.com/p/B7UL4IGIKZP/,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.instagram.com/p/B7UBEZZFz9M/,66.0,miniliciousbyrachelle,318,1490,651,2020-01-14T20:02:39.000Z
8,https://www.instagram.com/p/B7UIYOhAExm/,17.0,latandabarmid,391,1214,508,2020-01-14T21:06:31.000Z
9,https://www.instagram.com/p/B7UEy0QHk0S/,18.0,ealingeats,210,188,342,2020-01-14T20:35:12.000Z
10,https://www.instagram.com/p/B7UEhglCvUn/,11.0,weedzpicture,52,154,14,2020-01-14T20:32:50.000Z
11,https://www.instagram.com/p/B7UKdFTA2qe/,28.0,chirazfitness,247,1141,676,2020-01-14T21:24:40.000Z
12,https://www.instagram.com/p/B7UCnhEn7ph/,26.0,pieceofbean,16,118,160,2020-01-14T20:16:11.000Z
13,https://www.instagram.com/p/B7UK7lYgEzP/,46.0,healthyliving_marie,154,1530,1497,2020-01-14T21:28:50.000Z
14,https://www.instagram.com/p/B7UCK0Bo1O0/,NaN,NaN,NaN,NaN,NaN,NaN


In [228]:
#use df.iloc[a:, :] to start from a new position
for (index, row) in df.iterrows():
    link = df.loc[index,'Links']
    IB.browser.get(link)
    time.sleep(0.5)
    try:
        for (key, value) in get_post_info(IB).items():

            df.at[index, key] = value
    except: print('No Post Found')
    time.sleep(2)
    #if index>20: break  #for testing

No Post Found


KeyboardInterrupt: 

In [227]:
df

,Links,likes,username,num_posts,followers,following,posttime
0,https://www.instagram.com/p/B7UKNmZCVxf/,30.0,gianluca.melillo.80,34,392,257,2020-01-14T21:22:33.000Z
1,https://www.instagram.com/p/B7UD7UMlLkK/,16.0,mizz_martha,2253,404,526,2020-01-14T20:27:37.000Z
2,https://www.instagram.com/p/B7UAvb9pO1J/,21.0,maestrosdelfuego,87,1588,367,2020-01-14T19:59:47.000Z
3,https://www.instagram.com/p/B7UDiEjhA05/,62.0,paulawdrodzedocelu,94,1541,1456,2020-01-14T20:24:11.000Z
4,https://www.instagram.com/p/B7UB8Y-BgS8/,2702.0,wingsource,1081,51477,1805,2020-01-14T20:10:18.000Z
5,https://www.instagram.com/p/B7ULwqWCgTQ/,19.0,fassbender_fancy_official,689,3372,561,2020-01-14T21:36:04.000Z
6,https://www.instagram.com/p/B7UL4IGIKZP/,NaN,NaN,NaN,NaN,NaN,NaN
7,https://www.instagram.com/p/B7UBEZZFz9M/,66.0,miniliciousbyrachelle,318,1490,651,2020-01-14T20:02:39.000Z
8,https://www.instagram.com/p/B7UIYOhAExm/,17.0,latandabarmid,391,1214,508,2020-01-14T21:06:31.000Z
9,https://www.instagram.com/p/B7UEy0QHk0S/,18.0,ealingeats,210,188,342,2020-01-14T20:35:12.000Z


In [127]:
for rows in df.rows:
    print (rows)

AttributeError: 'DataFrame' object has no attribute 'rows'

In [23]:
links = IB.browser.find_elements_by_xpath("//a[@href]")

In [24]:
len(links)

60

In [34]:
actionChain = webdriver.ActionChains(IB.browser)
for i in range(400):
    
    actionChain.key_down(Keys.PAGE_DOWN).key_up(Keys.PAGE_DOWN).perform()
    time.sleep(0.5)
    links = IB.browser.find_elements_by_xpath("//a[@href]")
    for element in links:
        try:
            href = element.get_attribute('href')
            if r'/p/' in href and href not in all_links: all_links.add(href)
            print('Got link')
        except:
            print('Exception')

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

...

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
Got link
G

KeyboardInterrupt: 

In [35]:
len(all_links)

5454

In [36]:
list_of_links = list(all_links)

In [38]:
IB.browser.get(list_of_links[0])

In [40]:
likes_span = IB.browser.find_element_by_class_name('Nm9Fw')

In [44]:
likes_span.text

'59 likes'

In [45]:
likes_span.find_element_by_tag_name('span').text

'59'

In [ ]:
for element in links:
    if r'/p/' in element.get_attribute('href'):    print(element.get_attribute('href'))

In [18]:
for element in links:
    if r'/p/' in element.get_attribute('href'):    print(element.get_attribute('href'))

https://www.instagram.com/p/B7UFd6HFqrh/
https://www.instagram.com/p/B7UIy8mhAWF/
https://www.instagram.com/p/B7T8aMdJ1-s/
https://www.instagram.com/p/B7UGxUgJisI/
https://www.instagram.com/p/B7UGGmNnM9C/
https://www.instagram.com/p/B7UJiSBnp_S/
https://www.instagram.com/p/B7UBJ59nbc1/
https://www.instagram.com/p/B7TzMU1pP1f/
https://www.instagram.com/p/B7UIkexg2v8/
https://www.instagram.com/p/B7UMkGBpKos/
https://www.instagram.com/p/B7UMkBCAL9d/
https://www.instagram.com/p/B7UMj-dJbqc/
https://www.instagram.com/p/B7UMXIiHAyr/
https://www.instagram.com/p/B7UMPvkpFEf/
https://www.instagram.com/p/B7UMjqZAD_Z/
https://www.instagram.com/p/B7UMjc2JmCo/
https://www.instagram.com/p/B7UMjXHlFx7/
https://www.instagram.com/p/B7UMjUhoEJT/
https://www.instagram.com/p/B7UMjTbpp_S/
https://www.instagram.com/p/B7UMjQFnxLP/
https://www.instagram.com/p/B7UMjC0lKsq/
https://www.instagram.com/p/B7UMi-ho0vW/
https://www.instagram.com/p/B7UMi16lkcJ/
https://www.instagram.com/p/B7UL24XBbaK/
https://www.inst

In [ ]:
//*[@id="react-root"]/section/main/article/div[1]/div/div/div[1]/div[3]/a

In [5]:
IB.signIn()

In [24]:
for num, user in enumerate(followers[500:700]):
    result = IB.like_posts_by_user(user)
    print('User', num, result)
    if result == 'Block': break

Private account
User 0 False
User 1 True
User 2 True
User 3 True
Last post
User 4 False
User 5 True
User 6 True
User 7 True
User 8 True
User 9 True
User 10 True
User 11 True
User 12 True
User 13 True
User 14 True
User 15 True
User 16 True
User 17 True
User 18 True
User 19 True
User 20 True
User 21 True
User 22 True
User 23 True
User 24 True
Private account
User 25 False
User 26 True
No first post
User 27 False
Private account
User 28 False
User 29 True
User 30 True
No first post
User 31 False
User 32 True
Private account
User 33 False
User 34 True
No first post
User 35 False
Last post
User 36 False
User 37 True
User 38 True
User 39 True
User 40 True
User 41 True
User 42 True
User 43 True
Private account
User 44 False
User 45 True
User 46 True
User 47 True
User 48 True
User 49 True
User 50 True
User 51 True
User 52 True
No first post
User 53 False
Already liked
User 54 False
User 55 True
User 56 True
User 57 True
User 58 True
User 59 True
User 60 True
Last post
User 61 False
No first po

In [8]:
followers = IB.followerList('vidiiasf')

In [ ]:
'Done (Collected 2019/Dec) - heyimscott.x' #Great Trasnfer
'Done - tiagofdsalves'
'_ricardobentes_'
'carlosvictormaia'
'guyvanvliet'
'marcosveggas'



In [235]:
IB.like_posts_by_user(followers[202])

Unknown error


True

In [19]:
followers[300:500]

['https://www.instagram.com/brrunosexy/',
 'https://www.instagram.com/jockssonk2/',
 'https://www.instagram.com/yan_pablo.a/',
 'https://www.instagram.com/david_koroi/',
 'https://www.instagram.com/_.martemon._/',
 'https://www.instagram.com/luanperobelli/',
 'https://www.instagram.com/lucassouza15449/',
 'https://www.instagram.com/jonathas_philips/',
 'https://www.instagram.com/eversonferreeir/',
 'https://www.instagram.com/murilosantos16/',
 'https://www.instagram.com/aleoliveiro/',
 'https://www.instagram.com/marcosventtura/',
 'https://www.instagram.com/felipe_rammon/',
 'https://www.instagram.com/mcbernalfc/',
 'https://www.instagram.com/djguedes.ofc/',
 'https://www.instagram.com/wesleeyx_/',
 'https://www.instagram.com/andrealvesde8/',
 'https://www.instagram.com/juanalenk/',
 'https://www.instagram.com/_elielsoncosta/',
 'https://www.instagram.com/lahmatija/',
 'https://www.instagram.com/vbsjzkzjjz/',
 'https://www.instagram.com/theanotherfakeinthahouse/',
 'https://www.instagr

In [260]:
IB.reset_activity()

True

In [241]:
active_button = IB.browser.find_elements_by_css_selector('svg[aria-label=\'Activity Feed\']')
if bool(active_button): 
    active_button[0].click()

In [239]:
active_button[0].click()

IndexError: list index out of range

In [240]:
bool(active_button)

False

In [242]:
IB.like_posts_by_user(followers[202])

Unknown error


True

In [247]:
block_button = IB.browser.find_elements_by_xpath("//*[contains(text(), 'This action was blocked')]")

In [ ]:
This action was blocked. Please try again later. We restrict certain content and actions to protect our community. Tell us if you think we made a mistake.

In [248]:
bool(block_button)

False

In [249]:
IB.first_post()

ElementClickInterceptedException: Message: element click intercepted: Element <div class="_9AhH0"></div> is not clickable at point (672, 774). Other element would receive the click: <div class="_8Mwnh" role="dialog"></div>
  (Session info: chrome=79.0.3945.88)


In [305]:
followers[701:800]

['https://www.instagram.com/chelseazehrab/',
 'https://www.instagram.com/ahmedsarfraz4/',
 'https://www.instagram.com/harrylyonss/',
 'https://www.instagram.com/tempux/',
 'https://www.instagram.com/vaughn54136/',
 'https://www.instagram.com/jack728280/',
 'https://www.instagram.com/greenlos88/',
 'https://www.instagram.com/bb_mason26/',
 'https://www.instagram.com/jarivaaraniemi7/',
 'https://www.instagram.com/e_s_c_love/',
 'https://www.instagram.com/tucker_roper/',
 'https://www.instagram.com/edgitator/',
 'https://www.instagram.com/cc.holylight/',
 'https://www.instagram.com/dylanmurray1997/',
 'https://www.instagram.com/michicica/',
 'https://www.instagram.com/muzza22.5/',
 'https://www.instagram.com/l.x.uis/',
 'https://www.instagram.com/george_go19/',
 'https://www.instagram.com/ourrainbowworld/',
 'https://www.instagram.com/aye.itz.nizha.17/',
 'https://www.instagram.com/bex.harax/',
 'https://www.instagram.com/tw74s/',
 'https://www.instagram.com/absolutemessofcreation/',
 'ht

In [353]:
IB2 = InstagramBot2(IB.browser)

In [354]:
IB2.first_post()

True

In [355]:
IB2.like_post()

True